In [1]:
from cemba_data.bulk import prepare_mc_bulk
import subprocess
import os
import pandas as pd
import pathlib

In [2]:
# import os
# import sys

# # Check if CONDA_PREFIX environment variable is set
# if 'CONDA_PREFIX' in os.environ:
#     conda_prefix = os.environ['CONDA_PREFIX']
#     # Extract the environment name from the directory path
#     env_name = os.path.basename(conda_prefix)
#     print(f"Active Conda environment name: {env_name}")
# else:
#     print("No Conda environment is currently activated.")

In [3]:
#group_name = 'group10'

In [4]:
cwd_dir = os.getcwd()
_dir = '/data/tmp/ALLC_TABLE/'
chrom_size_path = '/ref/m3C/mm10.main.nochrM.nochrY.chrom.sizes'
outdir = f'{cwd_dir}/Celltype.Age'
allc_table = f'{_dir}/{group_name}.tsv'

NameError: name 'group_name' is not defined

In [ ]:
pathlib.Path('ALLC_FILES').mkdir(exist_ok=True)

## cp files

In [ ]:
allc_table = pd.read_csv(allc_table, header = None, sep = '\t')
allc_table[0] = allc_table[0].apply(lambda x: x.replace('/data','gs://ecker-rachel-amb-datasets'))

In [ ]:
all_allc_paths = allc_table[0].tolist()
allc_path_tbi = [f'{path}.tbi' for path in all_allc_paths]
with open(f'{group_name}.allc_paths.txt', 'w') as f:
    f.writelines(line + '\n' for line in all_allc_paths)

with open(f'{group_name}.allc_paths_tbi.txt', 'w') as f:
    f.writelines(line + '\n' for line in allc_path_tbi)

In [ ]:
if pathlib.Path('copy.finished').exists() == False:
    subprocess.run(f'gsutil -m cp -I < {group_name}.allc_paths.txt {cwd_dir}/ALLC_FILES', 
                    shell= True, stdout = subprocess.DEVNULL, stderr = subprocess.DEVNULL)
    subprocess.run(f'gsutil -m cp -I < {group_name}.allc_paths_tbi.txt {cwd_dir}/ALLC_FILES', 
                shell= True, stdout = subprocess.DEVNULL, stderr = subprocess.DEVNULL)

In [5]:
subprocess.run(f'touch copy.finished', shell= True, stdout = subprocess.DEVNULL, stderr = subprocess.DEVNULL)

CompletedProcess(args='touch copy.finished', returncode=0)

## generate new_allc table

In [13]:
allc_table[0] = allc_table[0].apply(lambda x: f'{cwd_dir}/' + 'ALLC_FILES/' + x.split('/')[-1])
allc_table.head()

,0,1
0,/home/qzeng_salk_edu/project/230825-merge-allc...,CA2-FC-IG-Glut.8wk
1,/home/qzeng_salk_edu/project/230825-merge-allc...,CA2-FC-IG-Glut.8wk
2,/home/qzeng_salk_edu/project/230825-merge-allc...,CA2-FC-IG-Glut.8wk
3,/home/qzeng_salk_edu/project/230825-merge-allc...,CA2-FC-IG-Glut.8wk
4,/home/qzeng_salk_edu/project/230825-merge-allc...,CA2-FC-IG-Glut.8wk


In [14]:
allc_table.to_csv(f'{cwd_dir}/{group_name}.tsv', header = None, sep = '\t', index = None)

## pre snakemake

In [15]:
allc_table_path = f'{cwd_dir}/{group_name}.tsv'
allc_table_path

'/home/qzeng_salk_edu/project/230825-merge-allc/group10.tsv'

In [2]:
prepare_mc_bulk(allc_table_path,
                outdir, 
                chrom_size_path,
                mch_context='CHN',
                mcg_context='CGN',
                bigwig_mch_bin_size=30,
                bigwig_mcg_bin_size=1,
                cpu_per_job=10,
                total_cpu=32)
             

NameError: name 'allc_table_path' is not defined

## parelle

In [17]:
command_path = f'{outdir}/qsub/snakemake_cmds.txt'
file_list = pd.read_csv(command_path, header = None)[0].tolist()

In [18]:
with open(f'{outdir}/qsub/snakemake_cmds_new.txt', "w") as file:
    for i, element in enumerate(file_list):
        file.write(element + "&" + '\n')
    file.write('wait')

In [19]:
subprocess.run(f'bash {outdir}/qsub/snakemake_cmds_new.txt', shell = True, stdout = subprocess.DEVNULL, stderr = subprocess.DEVNULL)

CompletedProcess(args='bash /home/qzeng_salk_edu/project/230825-merge-allc/Celltype.Age/qsub/snakemake_cmds_new.txt', returncode=0)

In [20]:
subprocess.run(f'gsutil -m cp -r {outdir}/* gs://ecker-rachel-amb-datasets/female-amb/merge-allc/CellType.Age', 
               shell = True, stdout = subprocess.DEVNULL, stderr = subprocess.DEVNULL)

CompletedProcess(args='gsutil -m cp -r /home/qzeng_salk_edu/project/230825-merge-allc/Celltype.Age/* gs://ecker-rachel-amb-datasets/female-amb/merge-allc/CellType.Age', returncode=0)

In [21]:
subprocess.run(f'rm {cwd_dir}/ALLC_FILES -rf', shell = True, stdout = subprocess.DEVNULL, stderr = subprocess.DEVNULL)

CompletedProcess(args='rm /home/qzeng_salk_edu/project/230825-merge-allc/ALLC_FILES -rf', returncode=0)